In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a poet who specializes in writing imaginative and metaphorical poems about programming languages."
        ),
        ("human", "Write a poem about the programming language {language}."),
    ]
)

poet_chain = poet_prompt | chat

In [2]:
explain_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a literary analyst who explains poems with clarity. "
        ),
        ("human", "Explain the meaning of this poem:\n\n{poem}"),
    ]
)


explain_chain = explain_prompt | chat


final_chain = {"poem": poet_chain} | explain_chain

final_chain.invoke({"language": "python"})

Ode to Python

Python does not bite; it purrs through the console, a green snake laying along the cursor’s path,
inviting you to speak in plain speech, to breathe in tabs and spaces as if they were air.

Indentation is breath, not punishment—whitespace becomes gravity,
pulling meaning toward the surface, a quiet sun on a chalkboard.

The Zen hums in the background: simple is better than complex, readability a gentle law,
and every line wears its intention like a tidy scarf in a winter wind.

Guido’s whisper guides the thread: a name stitched to a serpent, a promise
that code can be a poem, a map you can hold in a palm.

Batteries included, a forest of modules humming like bees in a pantry of keys:
standard library, a pantry where recipes stand ready to summon doors.

Duck typing glides by: if it quacks like a duck, we trust the voice and not the bill,
a form of trust that lets function and shape sing together on the same page.

The GIL lingers in the doorway, a polite gatekeeper of sin

AIMessageChunk(content='This poem is a loving, multilayered homage to the Python programming language. It uses the figure of a serpent to personify Python and blends technical concepts with lyrical imagery to celebrate the language’s philosophy, culture, and practical power. Here’s a guided reading of its meaning and key motifs.\n\nOverall meaning\n- Python is portrayed as approachable, flexible, and almost magical in how it helps you think and build. The language is depicted not just as a tool, but as a partner in learning and creating.\n- The poem interweaves Python’s design principles (simplicity, readability, and explicit structure) with cultural touchstones (Guido van Rossum, Monty Python, Zen of Python) to show how language, community, and philosophy inform the craft of programming.\n- It also acknowledges both the opportunities and limits of Python: easy entry and elegance on one hand, and technical realities (the GIL, multiprocessing) on the other, as well as the ongoing urge t